In [1]:
import json

data = [json.loads(d) for d in open("openvivqa_submission.jsonl")]

In [2]:
data[0]

{'940': '940',
 'question': '<image>\nngười dân không được tụ tập quá bao nhiêu người ?',
 'answer': '10 người',
 'predicted': 'không'}

In [5]:
import os
from pathlib import Path

from langchain.globals import set_llm_cache
from langchain.prompts import ChatPromptTemplate
from langchain_community.cache import SQLiteCache
from langchain_core.output_parsers import StrOutputParser
from tqdm import tqdm

from vividbot.data.task.vivid_instruct_65k.utils.llms import LLM

os.makedirs(f"{Path.home()}/.cache", exist_ok=True)
set_llm_cache(SQLiteCache(database_path=f"{Path.home()}/.cache/.langchain.db"))


def get_generate_qa_pairs_chain():
  return (
    ChatPromptTemplate.from_messages(
      [
        (
          "system",
          "Bạn là trợ lý đánh giá về độ chính xác của câu trả lời của một hệ thống AI. "
          "Bạn được cung cấp một câu hỏi, một câu trả lời đúng và một câu trả lời dự đoán. "
          "Nhiệm vụ của bạn là đánh giá xem câu trả lời dự đoán có đúng hay không. "
          "Nếu câu trả lời dự đoán đúng, hãy trả lời 1. "
          "Nếu câu trả lời dự đoán sai, hãy trả lời 0. "
          "Chỉ cần trả lời 0 hoặc 1 mà không cần giải thích. "
          "Ví dụ: Câu hỏi: Ai là người đầu tiên đặt chân lên mặt trăng? "
          "Đáp án đúng: Neil Armstrong "
          "Câu trả lời: Neil Armstrong"
          "Trả lời: 1"
          "Ví dụ 2: Câu hỏi: Ai là người đầu tiên đặt chân lên mặt trăng? "
          "Đáp án đúng: Neil Armstrong "
          "Câu trả lời: Buzz Aldrin"
          "Trả lời: 0",
        ),
        (
          "human",
          "Câu hỏi: {question}\n\nĐáp án đúng: {answer}Câu trả lời: {predicted}",
        ),
      ]
    )
    | LLM
    | StrOutputParser()
  )


scores = []
for d in tqdm(data):
  chain = get_generate_qa_pairs_chain()
  score = float(
    chain.invoke(
      {
        "question": d["question"].replace("<image>", "").strip(),
        "answer": d["answer"],
        "predicted": d["predicted"],
      }
    )
  )
  scores.append({**d, "score": score})

In [7]:
correct = sum(s["score"] for s in scores)
total = len(scores)
accuracy = correct / total
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.19


In [9]:
with open("openvivqa_scores.jsonl", "w") as f:
  for s in scores:
    f.write(json.dumps(s, ensure_ascii=False) + "\n")

In [10]:
corrects = [s for s in scores if s["score"] == 1]

In [11]:
len(corrects)

679

In [36]:
corrects[15]

{'12516': '12516',
 'question': '<image>\nnhà hàng ngưu phồn có mấy tầng ?',
 'answer': 'nhà hàng có hai tầng',
 'predicted': 'nhà hàng ngưu phồn có hai tầng',
 'score': 1.0}